# Tutorial 15: Storage Backends & Compression

Learn how to:
- Configure Tensorus to use different storage backends
- Enable compression presets and quantization
- Inspect persisted artifacts

The notebook demonstrates environment configuration patterns with demo fallbacks.

In [ ]:
import os
from tutorial_utils import ping_server, ensure_dataset, ingest_tensor, pretty_json
import numpy as np

API = "http://127.0.0.1:7860"
SERVER = ping_server(API)
print(f"\ud83d\udce1 Tensorus server available: {SERVER}")

# Show relevant environment variables
for key in ["TENSORUS_TENSOR_STORAGE_PATH", "TENSORUS_STORAGE_BACKEND", "TENSORUS_COMPRESSION_PRESET"]:
    print(f"{key} = {os.environ.get(key, 'not set')}")


## Local filesystem persistence
When `TENSORUS_TENSOR_STORAGE_PATH` is set to a directory path, Tensorus persists datasets as `.pt` files.

In [ ]:
DATASET = "storage_demo"
tensor = np.random.rand(16, 16).astype("float32")

if SERVER:
    ensure_dataset(DATASET, API)
    result = ingest_tensor(DATASET, tensor, {"description": "local persistence demo"}, API)
else:
    result = {"demo_mode": True, "record_id": "demo-1"}

print(pretty_json(result))


## Switching to S3 or PostgreSQL
To use remote backends, set environment variables before starting the server. For example:

```bash
export TENSORUS_STORAGE_BACKEND=postgres
export TENSORUS_POSTGRES_HOST=localhost
export TENSORUS_POSTGRES_USER=tensorus_user
export TENSORUS_POSTGRES_PASSWORD=...
```

or

```bash
export TENSORUS_TENSOR_STORAGE_PATH=s3://your-bucket/prefix
```

## Compression presets
Setting `TENSORUS_COMPRESSION_PRESET` or using `/compression/preset` endpoints enables LZ4/GZIP and quantization options.

In [ ]:
compression_payload = {
    "preset": "balanced",
    "quantization": "int8"
}

if SERVER:
    try:
        compress_resp = requests.post(f"{API}/compression/preset", json=compression_payload, timeout=15)
        compress_resp.raise_for_status()
        compress_result = compress_resp.json()
    except requests.RequestException as exc:
        compress_result = {"error": str(exc), "demo_mode": True}
else:
    compress_result = {"demo_mode": True, "message": "Compression preset applied (simulated)."}

print(pretty_json(compress_result))
